# Get data from excel using Pandas

- loop through rows and create observations for each one and validate and save

In [71]:
import pandas as pd
from pathlib import Path

source_path = '/Users/ehaas/Documents/FHIR/Davinci-CDEX/my-notes/attachment example data.xlsx'
# file_name = 'prov_data.csv'#'a1c_data.csv' #'wt_data.csv'
path = Path.cwd() / source_path

#df = pd.read_('wt_data.csv', skip_blank_lines=True)
df = pd.read_excel(path,)
df = df.astype(str)
#df.dropna(how="all", inplace=True)
df.head(3)

,AttachmentType,ClaimId,ItemId,OrganizationId,ProviderId,MemberId,Attachment,Attachment.1
0,0..1,0..1,0..*,0..1,0..1,0..1,0..*,0..*
1,code,Identifier,Identifier,Identifier,Identifier,Identifier,resource,resource
2,unsolicited-attachments,http://example.org/cdex/claim-ids|11234657,http://example.org/cdex/item-ids|1,http://hl7.org/fhir/sid/us-npi|1407071236,http://hl7.org/fhir/sid/us-npi|9941339108,http://example.org/cdex/member-ids|234567,fhir_templates/docref1.yml,fhir_templates/docref2.yml


In [85]:
from fhir.resources import construct_fhir_element
from fhir.resources.fhirabstractmodel import ValidationError
from pathlib import Path
from fhir.resources.documentreference import DocumentReference
from utils import validate
from yaml import load as y_load, dump as y_dump
from json import dumps, loads

def get_type(parameter, attachment_type):
    
    type_coding = construct_fhir_element(
    'Coding',dict(
        system = 'http://terminology.hl7.org/CodeSystem/v2-0203',
        code = 'FILL' if attachment_type in ['solicited-attachments','prior-authorization'] else 'PLAC',
        display = 'Filler Identifier' if attachment_type in ['solicited-attachments','prior-authorization'] else 'Placer Identifier',
        )
    )
    
    if parameter in ['ClaimId', 'ItemId']:
           r_cc = construct_fhir_element(
            'CodeableConcept',dict(
            coding = [type_coding],
            )
        )
    else:
        return
    return r_cc


def get_dr(path):
    dr = DocumentReference.parse_file(path)
    print(help(dr))
    timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
    tzinfo = timezone(timedelta(hours=timezone_offset))
    f_now = datetime.now(tzinfo)
    print(f_now)

In [90]:
r_obj = construct_fhir_element('Parameters', {})
r_obj.parameter = []
print('-'*80)
for i_name,i_value in df.iteritems():
    print(i_name,i_value[0],i_value[1], df.AttachmentType[2])
    if i_value[1] == 'resource':
        param = construct_fhir_element(
            'ParametersParameter', {
                'name': i_name,
                'resource' :  DocumentReference.parse_file(i_value[2]) # get direct from yaml file
            }
        )
    elif i_value[1] == 'Identifier':         
        param = construct_fhir_element(
            'ParametersParameter', {
                'name': i_name,
                f'value{i_value[1]}':construct_fhir_element(i_value[1], dict(
                        system = i_value[2].split('|')[0] if '|' in i_value[2] else None,
                        value = i_value[2].split('|')[1] if '|' in i_value[2] else i_value[2],
                        type = get_type(i_name,df.AttachmentType[2])
                    )
                )         
            }
        )
    else:
            param = construct_fhir_element(
                'ParametersParameter', {
                    'name':i_name,
                    f'value{i_value[1].capitalize()}': i_value[2]    
                }
            )

#     print(param.yaml())
    r_obj.parameter.append(param)
print('-'*80)

# validate(r_obj) # no endpoint to validate!
# print('-'*80)
# print(r_obj.json(indent=2))
# instead of save directly as yaml need to round trip to save datetime properly for fhir

r_json = r_obj.json()
r_dict = loads(r_json)
r_yaml = y_dump(r_dict, sort_keys=False)
     
out_path = '/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/includes-yaml/cdex-unsolicited-attachment-example1-body.yml'
path = Path() / out_path
print(f'writing to {path}')
path.write_text(r_yaml)

--------------------------------------------------------------------------------
AttachmentType 0..1 code unsolicited-attachments
ClaimId 0..1 Identifier unsolicited-attachments
ItemId 0..* Identifier unsolicited-attachments
OrganizationId 0..1 Identifier unsolicited-attachments
ProviderId 0..1 Identifier unsolicited-attachments
MemberId 0..1 Identifier unsolicited-attachments
Attachment 0..* resource unsolicited-attachments
Attachment.1 0..* resource unsolicited-attachments
--------------------------------------------------------------------------------
writing to /Users/ehaas/Documents/FHIR/Davinci-CDEX/input/includes-yaml/cdex-unsolicited-attachment-example1-body.yml


273762